# Part I

## Getting Started

In [ ]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (20,10)
plt.style.use('ggplot')

# Matplotlib debe registrarse manualmente debido a un conflicto entre Pandas y Prophet
pd.plotting.register_matplotlib_converters()

## Lectura de datos

In [ ]:
sales_df = pd.read_csv(r'\prophet_ejemplo/retail_sales.csv', index_col='date', parse_dates=True)
sales_df.head()

## Preparar los datos para Prophet

Para que prophet funcione, los nombres de las columnas deben ser cambiados a "ds" & "y". 

In [ ]:
df = sales_df.reset_index()
df.head()

Renombramos las columnas como lo requiere "Prophet" y aclaración:<br>
No le va mucho que el index sea de tipo "DateTime", le parece mejor que "ds" sea una columna y 
el index queda con los enteros que indican cada fila

In [ ]:
df.rename(columns={'date':'ds', 'sales':'y'}, inplace=True)
df.head()

Graficamos

In [ ]:
df.set_index('ds').y.plot().get_figure();

<b>Análisis:</b><br>
Se observa un efecto de estacionalidad con tendencia a crecimiento positivo (modelo aditivo: si bien hay crecimiento, las magnitudes son en esencia las mismas).

Facebook usa, para limpiar datos anómalos/outliers, la transformada logarítmica sobre el set de datos que tenemos disponible. 
Esta es una serie de datos para ejemplo y como tal es muy simple, pero este no es el mejor método (Para eso existe el EDA).

In [ ]:
df['y'] = np.log(df['y'])
df.head()

In [ ]:
df.set_index('ds').y.plot().get_figure();

mismo gráfico pero con escala distinta

# Parte II

## Ejecutar Prophet

Nota: La frecuencia de datos es mensual, por lo tanto puede aparecer un mensaje de aviso de Prophet indicando:
    Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
No pasa nada, dado que como los datos son mensuales. 

In [ ]:
model = Prophet()
model.fit(df)

Para el ejemplo, se quieren conocer los valores futuros a dos años. <br>
Dado que el periodo es de 1 mes, se agrega al conjunto de datos una cantidad de periodos igual a 24. 

In [ ]:
future = model.make_future_dataframe(periods=24, freq='m') 
future.tail()

Forecasting

In [ ]:
forecast = model.predict(future) #devuelve un dataframe

El dataframe obtenido contiene las siguientes columnas

In [ ]:
forecast.head()

De estas columnas solo nos interesan algunas:

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

## Graficando los resultados de Prophet

Prophet tiene el método para graficar: .plot().<br>
<ul>
<li>Datos originales: puntos negros</li>
<li>Modelo: linea azul</li>
<li>Error del pronóstico: Área azul sombreada</li>
</ul>

In [ ]:
model.plot(forecast);

## Visualizacion de los modelos de Prophet

En orden de crear un dataframe útil para poder visualizarlo con el conjunto original de datos, 
necesitamos combinar la salida del modelo de Prophet con el data set original, y luego construir manualmente
un grafico (chart) usando pandas y matplotlib.

Primero, que ambos tengan el mismo index:

In [ ]:
df.set_index('ds', inplace=True)
forecast.set_index('ds', inplace=True)

RECORDAR QUE LA ESCALAS DE LOS DATASET ES DISTINTA: df sin log, forecast esta en escala logaritmica

Ahora combinamos los datos originales con el modelo de pronostico

In [ ]:
viz_df = sales_df.join(forecast[['yhat', 'yhat_lower', 'yhat_upper']], how = 'outer')

In [ ]:
viz_df.head()

In [ ]:
viz_df['yhat_rescaled'] = np.exp(viz_df['yhat'])
viz_df.head()

In [ ]:
viz_df[['sales','yhat_rescaled']].plot().get_figure();

In [ ]:
sales_df.index = pd.to_datetime(sales_df.index) # nos aseguramos que el index es un datetime object.
connect_date = sales_df.index[-2] #seleccion desde la segunda fecha hasta la última. 

In [ ]:
mask = (forecast.index > connect_date)
predict_df = forecast.loc[mask]
predict_df.head()

Construimos un dataframe para usar en la visualización

In [ ]:
viz_df = sales_df.join(predict_df[['yhat', 'yhat_lower', 'yhat_upper']], how = 'outer')
viz_df['yhat_scaled'] = np.exp(viz_df['yhat'])

In [ ]:
viz_df.head()

In [ ]:
viz_df.tail()

Graficamos

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(viz_df.sales)
ax1.plot(viz_df.yhat_scaled, color='black', linestyle=':')
ax1.fill_between(viz_df.index, np.exp(viz_df['yhat_upper']), np.exp(viz_df['yhat_lower']), alpha=0.5, color='darkgray')
ax1.set_title('Sales (Orange) vs Sales Forecast (Black)')
ax1.set_ylabel('Dollar Sales')
ax1.set_xlabel('Date')

L=ax1.legend() #get the legend
L.get_texts()[0].set_text('Actual Sales') #change the legend text for 1st plot
L.get_texts()[1].set_text('Forecasted Sales') #change the legend text for 2nd plot
fig